# Lab-04-2 Loading Data

- Multivariate Linear Regression 복습
- "Minibatch" Gradient Descent 이론
- PyTorch Dataset and DataLoader 사용법

- 복잡한 머신러닝 모델을 학습하려면 엄청난 양의 데이터가 필요하다!
- 대부분의 데이터셋은 적어도 수십만 개의 데이터를 제공한다.
    - 영상 처리에서 유명한 데이터셋 : IMAGENET (14,197,122 개의 이미지)
    
- 엄청난 양의 데이터를 한번에 학습시킬 수 없다!
- 그렇다면 일부분의 데이터로만 학습하면 어떨까?

## Minibatch Gradient Descent
- 전체 데이터를 균일하게 나눠서 학습하자!

전체 데이터 -> Minibatch 1~5

- 업데이트를 좀 더 빠르게 할 수 있다.
- 전체 데이터를 쓰지 않아서 잘못된 방향으로 업데이트를 할 수도 있다.
- Batch gradient descent에 비해 스무스하게 줄어들지 않고 좀 변동이 있을 수 있다.

- 데이터셋을 미니배치로 쪼개는 데 사용하는 PyTorch Dataset, DataLoader 모듈에 대해 알아보자.

In [1]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y

dataset = CustomDataset()

* `torch.utils.data.Dataset` 상속
* `__len__()`
    * 이 데이터셋의 총 데이터 수
* `__getitem__()`
    * 어떠한 인덱스 `idx`를 받았을 때, 그에 상응하는 입출력 데이터 반환

## PyTorch DataLoader
- `torch.utils.data.DataLoader` 사용
- `batch_size=2`
    - 각 minibatch의 크기
    - 통상적으로 2의 제곱수로 설정한다. (16, 32, 64, 128, 256, 512, ...)
- `shuffle=True`
    - Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꾼다.

- `enumerate(dataloader)` : minibatch 인덱스와 데이터를 받음.
- `len(dataloader)` : 한 epoch 당 minibatch 개수

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 모델 초기화
model = nn.Linear(3, 1)

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print("Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}".format(
            epoch, nb_epochs, batch_idx+1, len(dataloader),
            cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 53503.109375
Epoch    0/20 Batch 2/3 Cost: 12074.336914
Epoch    0/20 Batch 3/3 Cost: 2988.027100
Epoch    1/20 Batch 1/3 Cost: 1685.880859
Epoch    1/20 Batch 2/3 Cost: 537.744263
Epoch    1/20 Batch 3/3 Cost: 167.021790
Epoch    2/20 Batch 1/3 Cost: 74.356964
Epoch    2/20 Batch 2/3 Cost: 7.075104
Epoch    2/20 Batch 3/3 Cost: 2.564416
Epoch    3/20 Batch 1/3 Cost: 5.479868
Epoch    3/20 Batch 2/3 Cost: 0.089473
Epoch    3/20 Batch 3/3 Cost: 8.633569
Epoch    4/20 Batch 1/3 Cost: 2.619488
Epoch    4/20 Batch 2/3 Cost: 2.284624
Epoch    4/20 Batch 3/3 Cost: 3.202001
Epoch    5/20 Batch 1/3 Cost: 2.426617
Epoch    5/20 Batch 2/3 Cost: 3.003861
Epoch    5/20 Batch 3/3 Cost: 2.844825
Epoch    6/20 Batch 1/3 Cost: 1.141958
Epoch    6/20 Batch 2/3 Cost: 2.662566
Epoch    6/20 Batch 3/3 Cost: 6.131699
Epoch    7/20 Batch 1/3 Cost: 2.045546
Epoch    7/20 Batch 2/3 Cost: 1.766446
Epoch    7/20 Batch 3/3 Cost: 4.948683
Epoch    8/20 Batch 1/3 Cost: 2.483003
Epoch 

## What's Next?
- 지금까지는 어떠한 숫자 하나를 예측하는 모델을 만들었다.
- 분류하는 모델은 어떻게 만들 수 있을까? (Feat. Decision Boundary)